In [7]:
import xml.etree.ElementTree as ET

# Load your KML file
kml_file_path = 'WRS-2_example.kml'

In [17]:
from lxml import etree as ET

# Load your KML file
kml_file_path = 'WRS-2_example.kml'
parser = ET.XMLParser(strip_cdata=False)  # Preserve CDATA
tree = ET.parse(kml_file_path, parser)
root = tree.getroot()

# Helper function to find the parent of an element
def find_parent(element, root):
    for parent in root.iter():
        if element in list(parent):
            return parent
    return None

# List the names of Placemarks you want to keep
placemarks_to_keep = ['15_26']  # Add all the names you want to keep here

# Find all Placemark elements
placemarks = root.findall('.//{http://www.opengis.net/kml/2.2}Placemark')

# Remove Placemark elements not in the keep list
for placemark in placemarks:
    name = placemark.find('{http://www.opengis.net/kml/2.2}name')
    if name is not None and name.text not in placemarks_to_keep:
        parent = find_parent(placemark, root)
        if parent is not None:
            parent.remove(placemark)

# Save the modified KML to a new file
tree.write('modified_kml_file.kml', pretty_print=True, xml_declaration=True, encoding='UTF-8')


### Land Cover Paths by every five Rows

In [18]:
# Example lines provided
land_cover = [
    "Row 1-5: Path 1-65, 150-175, 205-225",
    "Row 6-10: Path 1-65, 115-180, 225-233",
    "Row 11-15: Path 1-200, 215-233",
    "Row 16-20: Path 1-210, 230-233",
    "Row 21-25: Path 1-60, 70-80, 95-210",
    "Row 26-30: Path 1-50, 100-205",
    "Row 31-35: Path 10-50, 105-205",
    "Row 36-40: Path 10-45, 105-205",
    "Row 41-45: Path 10-40, 115-210",
    "Row 46-50: Path 1-30, 115-210",
    "Row 51-55: Path 1-20, 80-90, 110-135, 140-150, 160-210",
    "Row 56-60: Path 1-15, 105-155, 145-150, 160-210",
    "Row 61-65: Path 1-15, 105-155, 145-150, 160-200",
    "Row 66-70: Path 1-10, 80-120, 155-185",
    "Row 71-75: Path 1-5, 70-75, 80-85, 90-115, 155-185, 215-234",
    "Row 76-80: Path 80-115, 155-180, 215-233",
    "Row 81-85: Path 85-115, 165-180, 220-233",
    "Row 86-90: Path 70-75, 90-100, 220-233",
    "Row 91-95: Path 70-80, 225-233",
    "Row 96-100: Path 220-233",
    "Row 101-105: Path 209-225",
    "Row 106-110: Path 65-170, 210-225",
    "Row 111-115: Path 1-30, 60-185, 205-233",
    "Row 116-120: Path 1-35, 45-233",
    "Row 121-122: Path 1-233"
]

# Function to parse lines and generate placemark combinations
def parse_placemark_lines(lines):
    placemarks_dict = {}
    
    for line in lines:
        # Split the line by ':' to separate row and path sections
        row_section, path_section = line.split(':')
        
        # Get the row range
        row_start, row_end = [int(r) for r in row_section.strip().replace('Row ', '').split('-')]
        
        # Prepare the key for the dictionary based on row range
        dict_key = f'R{row_start}_R{row_end}_Placemarks'
        placemarks_dict[dict_key] = []
        
        # Split the path section by ',' to get individual path ranges
        path_ranges = path_section.strip().split(', ')
        
        # Iterate over each path range
        for path_range in path_ranges:
            path_start, path_end = [int(p) for p in path_range.replace('Path ', '').split('-')]
            
            # Generate all path_row combinations within the given range
            for row in range(row_start, row_end + 1):
                for path in range(path_start, path_end + 1):
                    placemark = f'{path}_{row}'
                    placemarks_dict[dict_key].append(placemark)
    
    return placemarks_dict

# Parse the example lines to get the placemark combinations
placemark_combinations = parse_placemark_lines(land_cover)

# Print all the array names (keys) in placemark_combinations and the number of elements in each array
scenes_count = 0
for array_name, placemarks in placemark_combinations.items():
    scenes_count += len(placemarks)
    print(f"{array_name}: {len(placemarks)} scenes")

print(f"Summary of Placemarks: {scenes_count} scenes")

R1_R5_Placemarks: 560 scenes
R6_R10_Placemarks: 700 scenes
R11_R15_Placemarks: 1095 scenes
R16_R20_Placemarks: 1070 scenes
R21_R25_Placemarks: 935 scenes
R26_R30_Placemarks: 780 scenes
R31_R35_Placemarks: 710 scenes
R36_R40_Placemarks: 685 scenes
R41_R45_Placemarks: 635 scenes
R46_R50_Placemarks: 630 scenes
R51_R55_Placemarks: 595 scenes
R56_R60_Placemarks: 615 scenes
R61_R65_Placemarks: 565 scenes
R66_R70_Placemarks: 410 scenes
R71_R75_Placemarks: 470 scenes
R76_R80_Placemarks: 405 scenes
R81_R85_Placemarks: 305 scenes
R86_R90_Placemarks: 155 scenes
R91_R95_Placemarks: 100 scenes
R96_R100_Placemarks: 70 scenes
R101_R105_Placemarks: 85 scenes
R106_R110_Placemarks: 610 scenes
R111_R115_Placemarks: 925 scenes
R116_R120_Placemarks: 1120 scenes
R121_R122_Placemarks: 466 scenes
Summary of Placemarks: 14696 scenes


### Divide Path ranges into groups of five Paths

In [19]:
# Adjusted function to create groups of 5 paths from a given path range string
# Only combine a smaller group with the previous group if it's adjacent to the previous group
def create_path_groups(path_range_str):
    path_groups = []
    # Remove the word 'Path' and split into individual ranges
    path_ranges = path_range_str.replace('Path ', '').split(', ')
    for path_range in path_ranges:
        start, end = map(int, path_range.split('-'))
        # Create groups of 5 within each range
        while start <= end:
            group_end = min(end, start + 4)  # Calculate the end of the current group
            path_groups.append(f"{start}-{group_end}")  # Append the current group
            start = group_end + 1  # Update the start for the next group

    # Combine smaller groups with the previous group if it's adjacent
    for i in range(len(path_groups) - 1, 0, -1):
        current_start, current_end = map(int, path_groups[i].split('-'))
        prev_start, prev_end = map(int, path_groups[i - 1].split('-'))
        if current_start == prev_end + 1 and (current_end - current_start) < 4:
            path_groups[i - 1] = f"{prev_start}-{current_end}"  # Extend the previous group
            del path_groups[i]  # Remove the current small group

    return path_groups

# Initialize an empty dictionary to store the path groups
path_groups_dict = {}

# Iterate over the land_cover list and create path groups for each line
for line in land_cover:
    # Split the line by ':' to separate row and path sections
    row_range, path_str = line.split(': ')
    # Extract the row range key, e.g., '1-5'
    row_key = row_range.split(' ')[1]

    # Create path groups for the given paths
    path_groups = create_path_groups(path_str)
    # Store the path groups in the dictionary using the row range as the key
    path_groups_dict[row_key] = path_groups
    print(f"'{row_key}': {path_groups}")


'1-5': ['1-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65', '150-154', '155-159', '160-164', '165-169', '170-175', '205-209', '210-214', '215-219', '220-225']
'6-10': ['1-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65', '115-119', '120-124', '125-129', '130-134', '135-139', '140-144', '145-149', '150-154', '155-159', '160-164', '165-169', '170-174', '175-180', '225-233']
'11-15': ['1-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65', '66-70', '71-75', '76-80', '81-85', '86-90', '91-95', '96-100', '101-105', '106-110', '111-115', '116-120', '121-125', '126-130', '131-135', '136-140', '141-145', '146-150', '151-155', '156-160', '161-165', '166-170', '171-175', '176-180', '181-185', '186-190', '191-195', '196-200', '215-219', '220-224', '225-233']
'16-20': ['1-5', '6-10', '11-15', '16-20', '21-25', '26-30', 

### Validating the number of scenes in each region

In [20]:
# Function to parse land cover lines into a dictionary mapping row ranges to path ranges
def parse_land_cover(land_cover_lines):
    land_cover_dict = {}
    for line in land_cover_lines:
        row_section, path_section = line.split(':')
        row_key = row_section.strip().replace('Row ', '')
        path_ranges = [path_range.strip().replace('Path ', '') for path_range in path_section.split(',')]
        land_cover_dict[row_key] = path_ranges
    return land_cover_dict

# Parse the land cover lines
land_cover_dict = path_groups_dict

print(land_cover_dict)

# Function to find the path ranges for a given row range key
def find_path_ranges(row_range, land_cover_dict):
    for key, path_ranges in land_cover_dict.items():
        if row_range in key:
            return path_ranges
    return []

# Function to count elements within specific path ranges for a given array of placemarks
def count_elements_in_path_ranges(placemarks, path_ranges):
    counts = []
    for path_range in path_ranges:
        path_start, path_end = map(int, path_range.split('-'))
        count = sum(1 for placemark in placemarks if path_start <= int(placemark.split('_')[0]) <= path_end)
        counts.append(count)
    return counts

scenes_count = 0
# Iterate over the placemark_combinations and count placemarks in the path ranges
for array_name, placemarks in placemark_combinations.items():
     # Extract row range from the array name and format it (e.g., '1-5' from 'R1_R5_Placemarks')
    row_range = array_name.replace('R', '').replace('_Placemarks', '')
    # Split on underscore and join with hyphen to get '1-5' format
    row_range = '-'.join(row_range.split('_'))
    path_ranges = find_path_ranges(row_range, land_cover_dict)
    counts = count_elements_in_path_ranges(placemarks, path_ranges)
    print(f"{array_name}: {counts}")
    scenes_count += sum(counts)

print(f"Summary of Placemarks: {scenes_count} scenes")

{'1-5': ['1-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65', '150-154', '155-159', '160-164', '165-169', '170-175', '205-209', '210-214', '215-219', '220-225'], '6-10': ['1-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65', '115-119', '120-124', '125-129', '130-134', '135-139', '140-144', '145-149', '150-154', '155-159', '160-164', '165-169', '170-174', '175-180', '225-233'], '11-15': ['1-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65', '66-70', '71-75', '76-80', '81-85', '86-90', '91-95', '96-100', '101-105', '106-110', '111-115', '116-120', '121-125', '126-130', '131-135', '136-140', '141-145', '146-150', '151-155', '156-160', '161-165', '166-170', '171-175', '176-180', '181-185', '186-190', '191-195', '196-200', '215-219', '220-224', '225-233'], '16-20': ['1-5', '6-10', '11-15', '16-20', '21-25', '26-3

### Get Placemarks for regions in each row group

In [21]:
# Function to generate placemarks for each path and row combination
def generate_placemarks_for_path_row(row_range, path_range):
    start_row, end_row = map(int, row_range.split('-'))
    start_path, end_path = map(int, path_range.split('-'))
    return [f"{path}_{row}" for path in range(start_path, end_path + 1) for row in range(start_row, end_row + 1)]

# Adjust the naming within the loop
placemark_combinations = {}
for row_range, path_groups in path_groups_dict.items():
    regions_dict = {}
    # Format '1-5' to 'R1_R5'
    row_range_formatted = 'R' + row_range.replace('-', '_R')
    for path_group in path_groups:
        # Generate a key for each path group within the row range
        region_key = f"{row_range_formatted}_P{path_group.replace('-', '_P')}"
        # Generate the list of placemarks for each path group
        regions_dict[region_key] = generate_placemarks_for_path_row(row_range, path_group)
    # Store the regions dictionary in placemark_combinations
    placemark_combinations[f"{row_range_formatted}_Regions"] = regions_dict

total_regions = 0 
# Print the count of elements in each regions dictionary
for regions_name, regions_dict in placemark_combinations.items():
    count = len(regions_dict) 
    total_regions += count 
    print(f"  {regions_name}: {count} regions") 
    #print(placemarks_array)

print(f"  Total placemarks: {total_regions}")

  R1_R5_Regions: 22 regions
  R6_R10_Regions: 27 regions
  R11_R15_Regions: 43 regions
  R16_R20_Regions: 43 regions
  R21_R25_Regions: 37 regions
  R26_R30_Regions: 31 regions
  R31_R35_Regions: 28 regions
  R36_R40_Regions: 27 regions
  R41_R45_Regions: 25 regions
  R46_R50_Regions: 25 regions
  R51_R55_Regions: 23 regions
  R56_R60_Regions: 24 regions
  R61_R65_Regions: 22 regions
  R66_R70_Regions: 16 regions
  R71_R75_Regions: 18 regions
  R76_R80_Regions: 15 regions
  R81_R85_Regions: 11 regions
  R86_R90_Regions: 5 regions
  R91_R95_Regions: 3 regions
  R96_R100_Regions: 2 regions
  R101_R105_Regions: 3 regions
  R106_R110_Regions: 24 regions
  R111_R115_Regions: 36 regions
  R116_R120_Regions: 44 regions
  R121_R122_Regions: 46 regions
  Total placemarks: 600


In [14]:


# Function to create a new KML document
def create_kml_doc():
    kml_element = ET.Element('kml', xmlns="http://www.opengis.net/kml/2.2")
    document_element = ET.SubElement(kml_element, 'Document')
    return ET.ElementTree(kml_element), document_element

In [15]:
from lxml import etree as ET
import os

# Define the namespaces as used in the original KML
namespaces = {
    None: "http://www.opengis.net/kml/2.2",
    'gx': "http://www.google.com/kml/ext/2.2",
    'kml': "http://www.opengis.net/kml/2.2",
    'atom': "http://www.w3.org/2005/Atom"
}

# Function to create a new KML document with the same namespaces as the original
def create_kml_doc(namespaces):
    kml_element = ET.Element(ET.QName(namespaces['kml'], "kml"), nsmap=namespaces)
    document_element = ET.SubElement(kml_element, ET.QName(namespaces['kml'], "Document"))
    return ET.ElementTree(kml_element), document_element

# Function to process a single batch of placemarks and add them to the KML
def process_batch(kml_file_path, placemarks_to_keep, array_name, region_name):
    parser = ET.XMLParser(strip_cdata=False)  # Preserve CDATA
    tree = ET.parse(kml_file_path, parser)
    root = tree.getroot()
    
    # Create a new KML document with the same namespaces
    new_tree, new_document = create_kml_doc(namespaces)

    # Transfer all required elements from the original document to the new one
    for elem in root.find("{http://www.opengis.net/kml/2.2}Document"):
        if elem.tag.endswith('Style') or elem.tag.endswith('StyleMap'):
            new_document.append(elem)

    # Find and add the Placemark elements that match the ones in placemarks_to_keep
    for placemark_name in placemarks_to_keep:
        xpath_query = f".//{{http://www.opengis.net/kml/2.2}}Placemark[{{http://www.opengis.net/kml/2.2}}name='{placemark_name}']"
        placemark_element = root.find(xpath_query)
        if placemark_element is not None:
            new_document.append(placemark_element)

    # Create directory if it does not exist
    directory = f'WRS-2_{array_name}'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save the new KML to a file within the directory
    modified_kml_file_path = os.path.join(directory, f"{region_name}.kml")
    new_tree.write(modified_kml_file_path, pretty_print=True, xml_declaration=True, encoding='UTF-8')
    return modified_kml_file_path

In [22]:
# Main script to iterate through each array in placemark_combinations in batches
def main():
    modified_files = []

    # Convert the range strings to tuples of integers for comparison
    start_range_tuple = tuple(map(int, '101-105'.split('-')))
    end_range_tuple = tuple(map(int, '101-105'.split('-')))

    for array_name, regions_array in placemark_combinations.items():
        # Convert array_name to a tuple of integers for comparison
        array_name_tuple = tuple(map(int, array_name.split('_Regions')[0].replace('R', '').split('_')))
        # Check if the current array_name is within the specified range
        if start_range_tuple <= array_name_tuple <= end_range_tuple:
            for region_name, placemarks_array in regions_array.items():
                # Process each batch
                modified_kml_file_path = process_batch('WRS-2_bound_world_0.kml', placemarks_array, array_name, region_name)
                modified_files.append(modified_kml_file_path)
                print(f"[{array_name}] Processed batch {region_name} with {len(placemarks_array)} placemarks.")

    return modified_files

# Call the main function to start the batch processing
# placemark_combinations should be defined and populated with the correct data prior to this
modified_kml_files = main()

[R101_R105_Regions] Processed batch R101_R105_P209_P213 with 25 placemarks.
[R101_R105_Regions] Processed batch R101_R105_P214_P218 with 25 placemarks.
[R101_R105_Regions] Processed batch R101_R105_P219_P225 with 35 placemarks.


### Validate Reduced KML files

In [37]:
def count_regions_in_directory(directory, file_type):
    # Dictionary to hold the counts of regions in each folder
    region_counts = {}

    # Iterate over each subdirectory in the given directory
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            # Count the number of .kml files in the subdirectory
            kml_count = len([file for file in os.listdir(folder_path) if file.endswith(file_type)])
            region_counts[folder] = kml_count

    return region_counts

# Define the path to the 'WRS-2_reduced_regions' directory
directory_path = 'WRS-2_reduced_regions'

# Get the counts of regions in each folder
region_counts = count_regions_in_directory(directory_path, '.kml')

# Print the counts
sum = 0
for folder, count in region_counts.items():
    sum += count
    print(f"{folder}: {count} regions")

print(f"Total number of regions: {sum} regions")

coordinates: 0 regions
WRS-2_R101_R105_Regions: 1 regions
WRS-2_R106_R110_Regions: 13 regions
WRS-2_R111_R115_Regions: 7 regions
WRS-2_R116_R120_Regions: 2 regions
WRS-2_R11_R15_Regions: 25 regions
WRS-2_R16_R20_Regions: 24 regions
WRS-2_R1_R5_Regions: 5 regions
WRS-2_R21_R25_Regions: 16 regions
WRS-2_R26_R30_Regions: 15 regions
WRS-2_R31_R35_Regions: 16 regions
WRS-2_R36_R40_Regions: 13 regions
WRS-2_R41_R45_Regions: 14 regions
WRS-2_R46_R50_Regions: 18 regions
WRS-2_R51_R55_Regions: 14 regions
WRS-2_R56_R60_Regions: 11 regions
WRS-2_R61_R65_Regions: 9 regions
WRS-2_R66_R70_Regions: 12 regions
WRS-2_R6_R10_Regions: 9 regions
WRS-2_R71_R75_Regions: 11 regions
WRS-2_R76_R80_Regions: 6 regions
WRS-2_R81_R85_Regions: 7 regions
WRS-2_R86_R90_Regions: 3 regions
WRS-2_R91_R95_Regions: 2 regions
WRS-2_R96_R100_Regions: 1 regions
Total number of regions: 254 regions


In [39]:
import csv
from lxml import etree

import re

def extract_coordinates_from_placemark(placemark):
    # Extract the CDATA content from the description tag
    description = placemark.find('.//{http://www.opengis.net/kml/2.2}description').text
    if not description:
        return None

    # Regular expression patterns to find the required coordinates
    lat_ll_pattern = r"<strong>LAT LL</strong>: ([\d.-]+)<br>"
    lon_ll_pattern = r"<strong>LON LL</strong>: ([\d.-]+)<br>"
    lat_ur_pattern = r"<strong>LAT UR</strong>: ([\d.-]+)<br>"
    lon_ur_pattern = r"<strong>LON UR</strong>: ([\d.-]+)<br>"

    # Extract coordinates using the patterns
    lat_ll = re.search(lat_ll_pattern, description)
    lon_ll = re.search(lon_ll_pattern, description)
    lat_ur = re.search(lat_ur_pattern, description)
    lon_ur = re.search(lon_ur_pattern, description)

    if lat_ll and lon_ll and lat_ur and lon_ur:
        # Return coordinates as (LL Lon, LL Lat, UR Lon, UR Lat)
        return (float(lon_ll.group(1)), float(lat_ll.group(1)), float(lon_ur.group(1)), float(lat_ur.group(1)))
    else:
        print("There's and issue with parsing the coordinates")
        return None

def process_kml_file(kml_file_path):
    tree = etree.parse(kml_file_path)
    root = tree.getroot()

    # Namespace map to handle KML's default namespace
    ns_map = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Extracting placemark data
    placemark_data = []
    for placemark in root.findall('.//kml:Placemark', ns_map):
        # Extract the name of the placemark
        placemark_name = placemark.find('.//kml:name', ns_map).text if placemark.find('.//kml:name', ns_map) is not None else 'Unknown'

        # Extract coordinates
        coordinates = extract_coordinates_from_placemark(placemark)
        if coordinates:
            # Append a tuple with placemark name and coordinates
            placemark_data.append((placemark_name,) + coordinates)

    return placemark_data

def main():
    base_directory = 'WRS-2_reduced_regions'
    output_directory = 'WRS-2_reduced_regions_coordinates'
    all_placemark_names = []  # List to store all placemark names

    for folder in os.listdir(base_directory):
        folder_path = os.path.join(base_directory, folder)
        output_path = os.path.join(output_directory, folder)
        if os.path.isdir(folder_path):
            print(f"Processing folder: {folder}")
            for file in os.listdir(folder_path):
                if file.endswith('.kml'):
                    kml_file_path = os.path.join(folder_path, file)
                    placemark_data = process_kml_file(kml_file_path)

                    # Add placemark names to the list
                    for data in placemark_data:
                        all_placemark_names.append(data[0])

                    # Prepare CSV file name and path
                    csv_file_name = file.replace('.kml', '_coordinates.csv')
                    csv_file_path = os.path.join(output_path, csv_file_name)

                    # Check if output directory exists, if not, create it
                    if not os.path.exists(output_path):
                        os.makedirs(output_path)

                    # Writing to CSV
                    with open(csv_file_path, 'w', newline='') as csvfile:
                        csvwriter = csv.writer(csvfile)
                        # Write headers
                        csvwriter.writerow(['Placemark', 'LL Lon', 'LL Lat', 'UR Lon', 'UR Lat'])
                        # Write placemark data
                        csvwriter.writerows(placemark_data)
                    print(f"  Processed {file} into {csv_file_name}")

    # Writing the list of all placemark names to a CSV file
    with open(os.path.join(output_directory, 'WRS-2_reduced_regions_all_placemarks.csv'), 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Placemark Name'])
        for name in all_placemark_names:
            csvwriter.writerow([name])
    print("Processed all files and saved all placemark names to 'all_placemark_names.csv'")

if __name__ == "__main__":
    main()

Processing folder: coordinates
Processing folder: WRS-2_R101_R105_Regions
  Processed R101_R105_P214_P218.kml into R101_R105_P214_P218_coordinates.csv
Processing folder: WRS-2_R106_R110_Regions
  Processed R106_R110_P100_P104.kml into R106_R110_P100_P104_coordinates.csv
  Processed R106_R110_P105_P109.kml into R106_R110_P105_P109_coordinates.csv
  Processed R106_R110_P115_P119.kml into R106_R110_P115_P119_coordinates.csv
  Processed R106_R110_P120_P124.kml into R106_R110_P120_P124_coordinates.csv
  Processed R106_R110_P125_P129.kml into R106_R110_P125_P129_coordinates.csv
  Processed R106_R110_P135_P139.kml into R106_R110_P135_P139_coordinates.csv
  Processed R106_R110_P145_P149.kml into R106_R110_P145_P149_coordinates.csv
  Processed R106_R110_P150_P154.kml into R106_R110_P150_P154_coordinates.csv
  Processed R106_R110_P160_P164.kml into R106_R110_P160_P164_coordinates.csv
  Processed R106_R110_P220_P225.kml into R106_R110_P220_P225_coordinates.csv
  Processed R106_R110_P75_P79.kml in

In [38]:
# Define the path to the 'WRS-2_reduced_regions' directory
directory_path = 'WRS-2_reduced_regions_coordinates'

# Get the counts of regions in each folder
region_counts = count_regions_in_directory(directory_path, '.csv')

# Print the counts
sum = 0
for folder, count in region_counts.items():
    sum += count
    print(f"{folder}: {count} regions")

print(f"Total number of regions: {sum} regions")

WRS-2_R101_R105_Regions: 1 regions
WRS-2_R106_R110_Regions: 13 regions
WRS-2_R111_R115_Regions: 7 regions
WRS-2_R116_R120_Regions: 2 regions
WRS-2_R11_R15_Regions: 25 regions
WRS-2_R16_R20_Regions: 24 regions
WRS-2_R1_R5_Regions: 5 regions
WRS-2_R21_R25_Regions: 16 regions
WRS-2_R26_R30_Regions: 15 regions
WRS-2_R31_R35_Regions: 16 regions
WRS-2_R36_R40_Regions: 13 regions
WRS-2_R41_R45_Regions: 14 regions
WRS-2_R46_R50_Regions: 18 regions
WRS-2_R51_R55_Regions: 14 regions
WRS-2_R56_R60_Regions: 11 regions
WRS-2_R61_R65_Regions: 9 regions
WRS-2_R66_R70_Regions: 12 regions
WRS-2_R6_R10_Regions: 9 regions
WRS-2_R71_R75_Regions: 11 regions
WRS-2_R76_R80_Regions: 6 regions
WRS-2_R81_R85_Regions: 7 regions
WRS-2_R86_R90_Regions: 3 regions
WRS-2_R91_R95_Regions: 2 regions
WRS-2_R96_R100_Regions: 1 regions
Total number of regions: 254 regions
